# Tutorial 37: OWA Portfolio Optimization

## 1. Downloading the data:

In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import warnings

warnings.filterwarnings("ignore")
pd.options.display.float_format = '{:.4%}'.format

# Date range
start = '2016-01-01'
end = '2019-12-30'

# Tickers of assets
assets = ['JCI', 'TGT', 'CMCSA', 'CPB', 'MO', 'APA', 'MMC', 'JPM',
          'ZION', 'PSA', 'BAX', 'BMY', 'LUV', 'PCAR', 'TXT', 'TMO',
          'DE', 'MSFT', 'HPQ', 'SEE', 'VZ', 'CNP', 'NI', 'T', 'BA']
assets.sort()

# Downloading data
data = yf.download(assets, start = start, end = end)
data = data.loc[:,('Adj Close', slice(None))]
data.columns = assets

[*********************100%%**********************]  25 of 25 completed


In [2]:
# Calculating returns

Y = data[assets].iloc[-300:,:].pct_change().dropna()

display(Y.head())

,APA,BA,BAX,BMY,CMCSA,CNP,CPB,DE,HPQ,JCI,...,NI,PCAR,PSA,SEE,T,TGT,TMO,TXT,VZ,ZION
Date,,,,,,,,,,,,,,,,,,,,,
2018-10-19,0.0475%,-0.8599%,-1.4332%,-3.0011%,0.1113%,1.2968%,3.4360%,-0.8764%,0.2945%,-0.7834%,...,0.6340%,-0.1823%,0.9185%,-0.7728%,1.1385%,-1.6075%,-1.1145%,-1.2872%,0.4575%,-0.8025%
2018-10-22,-1.9240%,-0.0786%,-0.6335%,-6.2984%,-0.6393%,-1.1024%,0.0528%,-0.3221%,1.1325%,-0.8199%,...,-0.8662%,0.4483%,-1.6953%,-2.8972%,-0.6085%,1.4753%,-0.6075%,-0.8634%,0.1457%,-3.4490%
2018-10-23,-3.6570%,-1.6658%,-0.4202%,-0.4521%,-0.2797%,-0.5034%,0.1844%,-3.9948%,-0.7051%,-0.2449%,...,0.4765%,-5.1239%,0.5342%,-0.0321%,1.0713%,-0.6729%,-1.0807%,-1.8308%,4.0560%,4.0353%
2018-10-24,-4.5501%,1.3141%,-1.8042%,-3.5933%,-4.2917%,0.8674%,0.9995%,-4.1109%,-3.6759%,-3.7139%,...,3.5178%,-4.2683%,1.5636%,-1.3479%,-8.0557%,-0.4839%,-1.2403%,-4.2187%,0.3671%,-3.3065%
2018-10-25,0.4741%,2.5715%,0.5186%,0.7782%,5.0411%,-0.5733%,-1.1719%,2.1585%,3.1656%,2.3271%,...,-1.0309%,0.4914%,0.5081%,0.9109%,-1.2516%,1.8962%,4.3662%,1.3799%,-1.7242%,3.3538%


## 2. Estimating OWA Portfolios

### OWA 포트폴리오 최적화 설명

OWA(Ordered Weighted Averaging) 포트폴리오 모델은 다양한 리스크 측정치와 목표 함수를 사용하여 포트폴리오를 최적화하는 방법론입니다. 이 방법론은 자산의 수익률을 특정 순서에 따라 가중치를 부여하여 평균화하는 방식으로, 자산의 리스크 및 수익률 분포의 다양한 특성을 고려할 수 있습니다.

### OWA 모델의 개요

OWA 모델은 다양한 순위 기반 가중치(weighted averaging)를 사용하여 포트폴리오 리스크를 측정합니다. 이는 자산 수익률의 순서 통계를 기반으로 하여, 특정 수익률 구간에 더 큰 중요성을 부여하거나, Tail 리스크를 강조할 수 있습니다.

### 주요 특징

1. **유연한 리스크 측정**: OWA 모델은 다양한 리스크 측정치를 지원하여, 투자자의 리스크 선호도에 따라 포트폴리오를 최적화할 수 있습니다.
2. **순서 기반 가중치**: 자산 수익률의 순서 통계에 따라 가중치를 부여하여 리스크를 측정합니다.
3. **Tail 리스크 고려**: Tail Gini Range와 같은 리스크 측정치를 사용하여 포트폴리오의 극단적 리스크를 관리할 수 있습니다.

### OWA 포트폴리오 최적화 단계

1. **데이터 준비**
2. **포트폴리오 객체 생성**
3. **기대 수익률 및 공분산 행렬 계산**
4. **최적화 매개변수 설정**
5. **최적 포트폴리오 계산**
6. **결과 출력**

### 예제 코드

아래 예제 코드는 다양한 OWA 포트폴리오를 최적화하는 방법을 보여줍니다.

```python
import numpy as np
import pandas as pd
import riskfolio as rp
import mosek

# 1. 데이터 준비
np.random.seed(42)
Y = np.random.randn(100, 10)  # 100개의 샘플과 10개의 자산
asset_names = [f'Asset {i}' for i in range(1, 11)]
returns = pd.DataFrame(Y, columns=asset_names)

# 2. 포트폴리오 객체 생성
port = rp.Portfolio(returns=returns)

# 3. 기대 수익률 및 공분산 행렬 계산
method_mu = 'hist'  # 역사적 데이터를 사용하여 기대 수익률 추정
method_cov = 'hist'  # 역사적 데이터를 사용하여 공분산 행렬 추정
port.assets_stats(method_mu=method_mu, method_cov=method_cov)

# 4. MOSEK 솔버 설정
port.solvers = ['MOSEK']
port.sol_params = {'MOSEK': {'mosek_params': {mosek.iparam.num_threads: 2}}}

# 최적화 매개변수 설정
model = 'Classic'  # 사용할 모델: Classic (historical), BL (Black Litterman), FM (Factor Model)
rm_list = ['MV', 'CVaR', 'TGini']  # 사용될 리스크 측정치 리스트
obj = 'Sharpe'  # 목표 함수: Sharpe 비율 최대화
hist = True  # 역사적 시나리오 사용 여부
rf = 0  # 무위험 이자율
l = 0  # 리스크 회피 계수 (유틸리티 목표 함수에서만 사용)

# 최적 포트폴리오 계산 및 결과 저장
results = {}

for rm in rm_list:
    w = port.optimization(model=model, rm=rm, obj=obj, rf=rf, l=l, hist=hist)
    results[rm] = w

# 6. 결과 출력
for rm in results:
    print(f"Optimal weights using {rm}:")
    display(results[rm].T)
```

### 코드 설명

1. **데이터 준비**:
   ```python
   np.random.seed(42)
   Y = np.random.randn(100, 10)
   asset_names = [f'Asset {i}' for i in range(1, 11)]
   returns = pd.DataFrame(Y, columns=asset_names)
   ```
   - 예시 데이터를 생성하고 자산 이름을 설정합니다.

2. **포트폴리오 객체 생성**:
   ```python
   port = rp.Portfolio(returns=returns)
   ```

   - `rp.Portfolio` 클래스를 사용하여 포트폴리오 객체를 생성합니다.

3. **기대 수익률 및 공분산 행렬 계산**:
   ```python
   method_mu = 'hist'
   method_cov = 'hist'
   port.assets_stats(method_mu=method_mu, method_cov=method_cov)
   ```

   - 역사적 데이터를 사용하여 기대 수익률 및 공분산 행렬을 추정합니다.

4. **MOSEK 솔버 설정**:
   ```python
   port.solvers = ['MOSEK']
   port.sol_params = {'MOSEK': {'mosek_params': {mosek.iparam.num_threads: 2}}}
   ```

   - MOSEK 솔버를 사용하도록 설정합니다. MOSEK 솔버는 높은 계산 능력을 제공하여 복잡한 최적화 문제를 해결하는 데 적합합니다.

5. **최적화 매개변수 설정 및 최적 포트폴리오 계산**:
   ```python
   model = 'Classic'
   rm_list = ['MV', 'CVaR', 'TGini']
   obj = 'Sharpe'
   hist = True
   rf = 0
   l = 0

   results = {}

   for rm in rm_list:
       w = port.optimization(model=model, rm=rm, obj=obj, rf=rf, l=l, hist=hist)
       results[rm] = w
   ```

   - `model`은 클래식 모델을 사용합니다.
   - `rm_list`는 다양한 리스크 측정치(예: 평균-분산, CVaR, Tail Gini Range)를 포함합니다.
   - 각 리스크 측정치에 대해 최적화를 수행하고, 결과를 저장합니다.

6. **결과 출력**:
   ```python
   for rm in results:
       print(f"Optimal weights using {rm}:")
       display(results[rm].T)
   ```

   - 각 리스크 측정치에 대해 최적화된 포트폴리오 가중치를 출력합니다.

### OWA 포트폴리오 최적화의 장점

1. **다양한 리스크 측정치 사용**: OWA 모델은 다양한 리스크 측정치를 사용하여 포트폴리오를 최적화할 수 있습니다. 이는 투자자의 리스크 선호도와 시장 조건에 따라 유연하게 대응할 수 있게 합니다.
2. **순서 기반 가중치 부여**: 자산 수익률의 순서 통계에 따라 가중치를 부여하여 리스크를 측정합니다. 이를 통해 특정 수익률 구간에 더 큰 중요성을 부여할 수 있습니다.
3. **Tail 리스크 관리**: Tail Gini Range와 같은 리스크 측정치를 사용하여 포트폴리오의 극단적 리스크를 효과적으로 관리할 수 있습니다.

### 결론

OWA 포트폴리오 모델은 다양한 리스크 측정치와 목표 함수를 사용하여 포트폴리오를 최적화할 수 있는 유연한 방법을 제공합니다. Tail Gini Range는 특히 Tail 리스크를 잘 반영하여 포트폴리오 최적화에 유용합니다. MOSEK 솔버를 사용하여 높은 계산 능력을 확보하고 복잡한 최적화 문제를 효율적으로 해결할 수 있습니다. 이 예제 코드를 통해 다양한 리스크 측정치에 대한 최적화된 포트폴리오 가중치를 계산하고 비교할 수 있습니다.

### 2.1 Comparing Classical formulations vs OWA formulations.

이번에는 두 가지 리스크 측정치(조건부 가치-위험(CVaR)과 최악 실현(WR 또는 Minimax 모델))에 대해 고전적 공식화와 OWA 공식화를 사용하여 최적 가중치를 비교할 것입니다.

In [3]:
import riskfolio as rp
import mosek

# Building the portfolio object
port = rp.Portfolio(returns=Y)

# Calculating optimum portfolio

# Select method and estimate input parameters:

method_mu='hist' # Method to estimate expected returns based on historical data.
method_cov='hist' # Method to estimate covariance matrix based on historical data.

port.assets_stats(method_mu=method_mu, method_cov=method_cov)

# Estimate optimal portfolios:

port.solvers = ['MOSEK'] # It is recommended to use mosek when optimizing GMD
port.sol_params = {'MOSEK': {'mosek_params': {mosek.iparam.num_threads: 2}}}
alpha = 0.05

port.alpha = alpha
model ='Classic' # Could be Classic (historical), BL (Black Litterman) or FM (Factor Model)
rms = ['CVaR', 'WR'] # Risk measure used, this time will be CVaR and Worst Realization
objs = ['MinRisk', 'Sharpe'] # Objective function, could be MinRisk, MaxRet, Utility or Sharpe
hist = True # Use historical scenarios for risk measures that depend on scenarios
rf = 0 # Risk free rate
l = 0 # Risk aversion factor, only useful when obj is 'Utility'

ws = pd.DataFrame([])
for rm in rms:
    for obj in objs:
        # Using Classical models
        w = port.optimization(model=model, rm=rm, obj=obj, rf=rf, l=l, hist=hist)
        # Using OWA model
        if rm == "CVaR":
            owa_w = rp.owa_cvar(len(Y), alpha=alpha)
        elif rm == 'WR':
            owa_w = rp.owa_wr(len(Y))
        w1 = port.owa_optimization(obj=obj, owa_w=owa_w, rf=rf, l=l)
        ws1 = pd.concat([w, w1], axis=1)
        ws1.columns = ['Classic ' + obj + ' ' + rm, 'OWA ' + obj + ' ' + rm]
        ws1['diff ' + obj + ' ' + rm] = ws1['Classic ' + obj + ' ' + rm] - ws1['OWA ' + obj + ' ' + rm]
        ws = pd.concat([ws, ws1], axis=1)

ws.style.format("{:.2%}").background_gradient(cmap='YlGn', vmin=0, vmax=1)

,Classic MinRisk CVaR,OWA MinRisk CVaR,diff MinRisk CVaR,Classic Sharpe CVaR,OWA Sharpe CVaR,diff Sharpe CVaR,Classic MinRisk WR,OWA MinRisk WR,diff MinRisk WR,Classic Sharpe WR,OWA Sharpe WR,diff Sharpe WR
APA,0.00%,0.00%,-0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,-0.00%,0.00%,0.00%,-0.00%
BA,3.79%,3.79%,-0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,-0.00%,0.00%,0.00%,-0.00%
BAX,0.00%,0.00%,-0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,-0.00%,0.00%,0.00%,-0.00%
BMY,9.47%,9.47%,0.00%,1.84%,1.84%,0.00%,12.88%,12.88%,-0.00%,0.00%,0.00%,0.00%
CMCSA,0.87%,0.87%,-0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,-0.00%,0.00%,0.00%,0.00%
CNP,9.42%,9.42%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,-0.00%,0.00%,0.00%,0.00%
CPB,6.94%,6.94%,0.00%,28.53%,28.53%,0.00%,0.00%,0.00%,0.00%,33.71%,33.71%,-0.00%
DE,0.00%,0.00%,-0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,-0.00%
HPQ,0.00%,0.00%,-0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,-0.00%,0.00%,0.00%,0.00%
JCI,0.00%,0.00%,-0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,-0.00%,0.00%,0.00%,-0.00%


보시다시피, 고전적 공식화와 OWA 공식화는 동일한 수익률을 제공합니다. 이는 두 문제가 동등하기 때문입니다.